# BMI Regression using a Dense Layer

In [42]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [43]:
df = pd.read_csv('data/diabetes.csv')

X = df.drop(['Outcome', 'BMI'], axis=1)
y = df['BMI']

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.FloatTensor(y_test.values)

In [45]:
class SimpleDenseNet(nn.Module):
    def __init__(self, input_dim):
        super(SimpleDenseNet, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return self.linear(x)

In [46]:
net = SimpleDenseNet(X_train.shape[1])
print(net)

SimpleDenseNet(
  (linear): Linear(in_features=7, out_features=1, bias=True)
)


In [47]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [48]:
num_epochs = 1000
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        outputs = net(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 437.1578
Epoch [200/1000], Loss: 158.6707
Epoch [300/1000], Loss: 119.5952
Epoch [400/1000], Loss: 115.4975
Epoch [500/1000], Loss: 112.8363
Epoch [600/1000], Loss: 110.4913
Epoch [700/1000], Loss: 108.3510
Epoch [800/1000], Loss: 106.3800
Epoch [900/1000], Loss: 104.5566
Epoch [1000/1000], Loss: 102.8645


In [49]:
net.eval()
with torch.no_grad():
    y_pred = net(X_test_tensor).squeeze().numpy()
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f'RMSE: {rmse:.4f}')
    print(f'R2 Score: {r2:.4f}')

RMSE: 7.4834
R2 Score: 0.2122
